<link rel="stylesheet" href="https://doc.splicemachine.com/zeppelin/css/zepstyles2.css" />

# Useful Database Utilities

This notebook describes a number of utilities that you may find useful. We'll expand this list over time.

All you need to do is click *Shift + Enter* to run each paragraph and see the results.

In [ ]:
%%sql 

call SYSCS_UTIL.SYSCS_GET_VERSION_INFO();

In [ ]:
%%sql 
select schemaname from sys.sysschemas;

In [ ]:
%%sql 

select t.TABLENAME, s.SCHEMANAME from SYS.SYSTABLES t, SYS.SYSSCHEMAS s where t.SCHEMAID = s.SCHEMAID;

In [ ]:
%%sql 
SELECT 
		   S.SCHEMANAME AS TABLE_SCHEM, 
		   T.TABLENAME AS TABLE_NAME, 
		   (CASE WHEN CONGLOMS.DESCRIPTOR.isUnique() THEN FALSE ELSE TRUE END) AS NON_UNIQUE, 
		   CONGLOMS.CONGLOMERATENAME AS INDEX_NAME, 
		   CAST (CONGLOMS.DESCRIPTOR.getKeyColumnPosition(COLS.COLUMNNUMBER) AS SMALLINT) AS ORDINAL_POSITION, 
		   COLS.COLUMNNAME AS COLUMN_NAME, 
		   CASE WHEN CONGLOMS.DESCRIPTOR.isAscending( 
				CONGLOMS.DESCRIPTOR.getKeyColumnPosition(COLS.COLUMNNUMBER)) THEN 'A' ELSE 'D' END AS ASC_OR_DESC, 
		   CONGLOMS.CONGLOMERATENUMBER AS CONGLOM_NO 
	FROM 
			 SYS.SYSCOLUMNS COLS
			,SYS.SYSTABLES T 
			,SYS.SYSCONGLOMERATES CONGLOMS 
			,SYS.SYSSCHEMAS S 
	WHERE T.TABLEID = CONGLOMS.TABLEID AND T.TABLEID = COLS.REFERENCEID 
	  AND T.SCHEMAID = S.SCHEMAID 
	  AND CONGLOMS.ISINDEX 
	  AND (CASE WHEN CONGLOMS.DESCRIPTOR IS NOT NULL THEN 
				CONGLOMS.DESCRIPTOR.getKeyColumnPosition(COLS.COLUMNNUMBER) ELSE 
				0 END) <> 0  
	ORDER BY TABLE_SCHEM, TABLE_NAME, INDEX_NAME, ORDINAL_POSITION

In [ ]:
%%sql 
SELECT  
		   S.SCHEMANAME AS TABLE_SCHEM, T.TABLENAME AS TABLE_NAME, 
		   COLS.COLUMNNAME AS COLUMN_NAME, 
		   CAST (CONGLOMS.DESCRIPTOR.getKeyColumnPosition(COLS.COLUMNNUMBER) AS SMALLINT) AS KEY_SEQ
	FROM  
			SYS.SYSTABLES T 
			, SYS.SYSSCHEMAS S 
			, SYS.SYSCONSTRAINTS CONS 
			, SYS.SYSPRIMARYKEYS KEYS 
			, SYS.SYSCONGLOMERATES CONGLOMS 
			, SYS.SYSCOLUMNS COLS 
	WHERE 
		  T.SCHEMAID = S.SCHEMAID AND	
		  T.TABLEID = COLS.REFERENCEID AND T.TABLEID = CONGLOMS.TABLEID AND 
		  CONS.TABLEID = T.TABLEID AND CONS.TYPE = 'P' AND 
		  CONS.CONSTRAINTID = KEYS.CONSTRAINTID AND 
		  (CASE WHEN CONGLOMS.DESCRIPTOR IS NOT NULL THEN 
				CONGLOMS.DESCRIPTOR.getKeyColumnPosition(COLS.COLUMNNUMBER) ELSE 
				0 END) <> 0 AND 
		  KEYS.CONGLOMERATEID = CONGLOMS.CONGLOMERATEID 

In [ ]:
%%sql 
call syscs_util.syscs_get_running_operations();

## Where to Go Next

You're now ready to learn about the [*The Life of a Query*](./d.%20The%20Life%20of%20a%20Query.ipynb) in the next notebook in this class.

## Related Documentation

For more information about useful utility procedures for working with your Splice Machine database, see the <a href="https://doc.splicemachine.com/sqlref_sysprocs_intro.html" target="_blank">System Procedures</a> section of our SQL Reference Manual.
